Nomes: Bel Cogo e Bruno da Siqueira Hoffmann.

Professor: Felipe de Morais.

Disciplina: Ciência de Dados e Big Data.

# Tarefa 5

O banco de dados utilizado para essa tarefa contém dados referentes a dados de transações, consistindo em 5 tabelas, sendo elas:

- sellers: Contém informações sobre os vendedores, incluindo o estado em que estão localizados.
- products: Detalha os produtos disponíveis no marketplace, incluindo categorias e pesos.
- orders: Fornece informações básicas sobre cada pedido, como registros de data/hora e contato do cliente.
- order_items: Agrupa os itens associados a cada pedido, incluindo detalhes de preço.
- order_reviews: Armazena as avaliações relacionadas a cada pedido, incluindo classificações numéricas e comentários em texto.

É possível verificar o ER do banco de dados abaixo:

![Diagrama ER.jpg](./Diagrama%20ER.jpg)

Para importar o banco de dados foi executado o comando abaixo:

In [1]:
import pandas as pd
import sqlite3

connection = sqlite3.connect('transactional_data.sqlite')

## Pergunta 1

Pergunta: Quais produtos são vendidos por quais sellers?

Resposta:

,review_id,order_id,review_score,review_comment_message
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela I...
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,aparelho eficiente. no site a marca do aparelh...
3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,"Vendedor confiável, produto ok e entrega antes..."
...,...,...,...,...
39457,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,Entregou dentro do prazo. O produto chegou em ...
39458,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,"O produto não foi enviado com NF, não existe v..."
39459,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,"Excelente mochila, entrega super rápida. Super..."
39460,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,Solicitei a compra de uma capa de retrovisor c...


## Questão 2

Pergunta: Quais os 5 produtos mais vendidos?

## Questão 3

Pergunta: Qual o estado com mais sellers? E qual é o valor total faturado?

## Questão 4

Pergunta: Qual seller tem mais pedidos com review ruim? E qual é a média de nota dele?

## Questão 5

Pergunta: Qual o produto com mais ordens? E quanto ele fatura?